In [1]:
# Importing necessary packages.
import pandas as pd
import csv
import numpy as np

In [2]:
# import dask.dataframe as dd

In [3]:
# from dask.distributed import Client, progress
# client = Client(n_workers=2, threads_per_worker=2, memory_limit='8GB')
# client

### Load NBER Categories

In [4]:
# f_name = "application.tsv"
# f_name = "cpc_group.tsv"
# f_name = "cpc_subgroup.tsv"
# f_name = "cpc_subsection.tsv"
# f_name = "cpc_current.tsv"
# f_name = "mainclass.tsv"
# f_name = "mainclass_current.tsv"
# f_name = "subclass.tsv"
# f_name = "subclass_current.tsv"
f_name = "nber.tsv"
dtypes = {'category_id': np.int8, 'subcategory_id': np.int8}
nber = pd.read_csv(f_name, delimiter="\t", dtype=dtypes, quoting = csv.QUOTE_NONNUMERIC)

In [5]:
# nber has unique patent ids
nber['patent_id'].nunique()

5105937

In [6]:
nber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5105937 entries, 0 to 5105936
Data columns (total 4 columns):
uuid              object
patent_id         object
category_id       int8
subcategory_id    int8
dtypes: int8(2), object(2)
memory usage: 87.6+ MB


In [7]:
nber.dtypes

uuid              object
patent_id         object
category_id         int8
subcategory_id      int8
dtype: object

In [8]:
nber.sample(5)

,uuid,patent_id,category_id,subcategory_id
682139,4t8bq1wyrdvymnsonkaejgv4c,5269449,6,68
2175847,fc4bww84z9f1d9n58ijl6ddp5,8828320,1,19
2715239,j52uymsic5b6s1tlwi7fr62vl,4035808,4,49
4523063,vwiupduj3ew4wx0rmcd0xb821,8740508,5,51
4640864,wqbx38tdxgb217fn87ubcrdjl,7839209,4,41


In [9]:
# nber[nber['patent_id'] == '4433370']

In [10]:
# important step
nber = nber.astype({'patent_id': 'str'})

### load patent files

In [11]:
f_name = "patent.tsv"
dtypes = {'num_claims': np.int16, 'withdrawn': np.float32}
patent = pd.read_csv(f_name, delimiter="\t", dtype=dtypes, parse_dates = ['date'])

/afs/crc.nd.edu/user/y/yniu/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
patent.head()

,id,type,number,country,date,abstract,title,kind,num_claims,filename,withdrawn
0,10000000,utility,10000000,US,2018-06-19,A frequency modulated (coherent) laser detecti...,Coherent LADAR using intra-pixel quadrature de...,B2,20,ipg180619.xml,NaN
1,10000001,utility,10000001,US,2018-06-19,The injection molding machine includes a fixed...,Injection molding machine and mold thickness c...,B2,12,ipg180619.xml,NaN
2,10000002,utility,10000002,US,2018-06-19,The present invention relates to: a method for...,Method for manufacturing polymer film and co-e...,B2,9,ipg180619.xml,NaN
3,10000003,utility,10000003,US,2018-06-19,The invention relates to a method for producin...,Method for producing a container from a thermo...,B2,18,ipg180619.xml,NaN
4,10000004,utility,10000004,US,2018-06-19,The present invention relates to provides a do...,"Process of obtaining a double-oriented film, c...",B2,6,ipg180619.xml,NaN


In [13]:
patent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7430873 entries, 0 to 7430872
Data columns (total 11 columns):
id            object
type          object
number        object
country       object
date          datetime64[ns]
abstract      object
title         object
kind          object
num_claims    int16
filename      object
withdrawn     float32
dtypes: datetime64[ns](1), float32(1), int16(1), object(8)
memory usage: 552.8+ MB


In [14]:
# patent['country'].unique()
patent = patent.astype({'id': 'str'})

In [15]:
# patent = patent[['id', 'type', 'date']]

In [16]:
patent.sample(5)

,id,type,number,country,date,abstract,title,kind,num_claims,filename,withdrawn
6657972,9922105,utility,9922105,US,2018-03-20,A method and an apparatus of maintaining data ...,Method and apparatus of maintaining data for o...,B2,25,ipg180320.xml,0.0
2421572,5662000,utility,5662000,US,1997-09-02,A detent spring for use in a rotatable grip a...,Detent spring for rotatable grip actuating system,A,19,pftaps19970902_wk35.zip,0.0
5484969,8741452,utility,8741452,US,2014-06-03,A secondary battery is disclosed. The secondar...,Secondary battery,B2,27,ipg140603.xml,0.0
4738439,7991442,utility,7991442,US,2011-08-02,Disclosed herein is a multiple opening and clo...,Multiple opening and closing type mobile commu...,B2,12,ipg110802.xml,0.0
5170513,8425345,utility,8425345,US,2013-04-23,A variable length golf club shaft is disclosed...,Variable length shaft,B2,14,ipg130423.xml,0.0


In [17]:
patent.groupby('type').count()

,id,number,country,date,abstract,title,kind,num_claims,filename,withdrawn
type,,,,,,,,,,
TVPP,3,3,3,3,3,3,3,3,3,3
defensive publication,509,509,509,509,0,509,509,509,509,509
design,645674,645674,645674,645674,22,645674,645674,645674,645674,630205
plant,27887,27887,27887,27887,27858,27887,27887,27887,27887,27311
reissue,19227,19227,19227,19227,18929,19227,19227,19227,19227,18965
statutory invention registration,2258,2258,2258,2258,2251,2258,2258,2258,2258,2257
utility,6735315,6735315,6735315,6735315,6734508,6735315,6735315,6735315,6735315,6580496


In [18]:
# patent[patent['id'] == 4433370]

### select only utility patents

In [19]:
patent = patent[patent['type'] == 'utility']

In [20]:
# patent[patent['id'].str.contains("4433370")]

In [21]:
patent = patent.merge(nber,left_on = "id", right_on = "patent_id", how= "left")

### 75% of patent have NBER categories, which is almost every patent before 2015

In [22]:
patent[~(patent['subcategory_id'].isnull())].shape[0]/ patent.shape[0]

0.7555037886127078

In [23]:
patent[patent['subcategory_id'].isnull()].sample(20)

,id,type,number,country,date,abstract,title,kind,num_claims,filename,withdrawn,uuid,patent_id,category_id,subcategory_id
6651811,9915971,utility,9915971,US,2018-03-13,There is provided a transmission apparatus inc...,Transmission apparatus,B2,5,ipg180313.xml,0.0,NaN,NaN,NaN,NaN
5940934,9200395,utility,9200395,US,2015-12-01,A method of applying stitching to an interior ...,Method for stitching vehicle interior componen...,B2,14,ipg151201.xml,1.0,NaN,NaN,NaN,NaN
661384,10666500,utility,10666500,US,2020-05-26,Techniques are described for avoiding traffic ...,Failure handling for active-standby redundancy...,B2,23,ipg200526.xml,0.0,NaN,NaN,NaN,NaN
373581,10376351,utility,10376351,US,2019-08-13,The invention discloses an implant. The implan...,Medical device and method of delivering the me...,B2,19,ipg190813.xml,0.0,NaN,NaN,NaN,NaN
6536063,9799442,utility,9799442,US,2017-10-24,A magnetic assembly includes a bobbin with two...,Magnetic core structures for magnetic assemblies,B1,17,ipg171024.xml,0.0,NaN,NaN,NaN,NaN
6675891,9940239,utility,9940239,US,2018-04-10,A set-associative cache memory includes a bank...,Counter-based victim selection in a cache memory,B1,12,ipg180410.xml,0.0,NaN,NaN,NaN,NaN
5843843,9102672,utility,9102672,US,2015-08-11,The present invention relates to modulators of...,Azaindole derivatives as CFTR modulators,B2,37,ipg150811.xml,0.0,NaN,NaN,NaN,NaN
102564,10103311,utility,10103311,US,2018-10-16,A system comprises a plurality of thermoelectr...,Flexible sink for a thermoelectric energy gene...,B2,18,ipg181016.xml,NaN,NaN,NaN,NaN,NaN
606314,10610992,utility,10610992,US,2020-04-07,A coolant distributor comprising a head having...,Automated interchangeable coolant distributor,B2,8,ipg200407.xml,0.0,NaN,NaN,NaN,NaN
6158868,9419706,utility,9419706,US,2016-08-16,Embodiments of the present invention include a...,"Single fiber bi-directional optical module, an...",B2,11,ipg160816.xml,0.0,NaN,NaN,NaN,NaN


In [24]:
patent['year'] = patent['date'].dt.year

### load citations 

In [26]:
f_name = "uspatentcitation.tsv"
# # dtypes = {'sequence': np.int16}
# # citation = pd.read_csv(f_name, delimiter="\t", dtype=dtypes)
citation = pd.read_csv(f_name, delimiter="\t", usecols = ['patent_id','citation_id','date'], dtype=dtypes)

In [27]:
# use sample for experiments // or using the file below for reading partial data
# .sample(frac=0.1) 

In [28]:
# read partial data for speed/ prototypring 

f_name = "uspatentcitation.tsv"

# Take every N-th (in this case 10th) row
n = 10

# Count the lines or use an upper bound
num_lines = sum(1 for l in open(f_name))

# The row indices to skip - make sure 0 is not included to keep the header!
skip_idx = [x for x in range(1, num_lines) if x % n != 0]

# Read the data
# citation = pd.read_csv(f_name, skiprows=skip_idx, delimiter="\t", usecols = ['patent_id','citation_id','date'], dtype=dtypes)

In [29]:
citation = citation.astype({'patent_id': 'str', 'citation_id': 'str'})

In [30]:
citation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111026439 entries, 0 to 111026438
Data columns (total 3 columns):
patent_id      object
citation_id    object
date           object
dtypes: object(3)
memory usage: 2.5+ GB


In [31]:
citation.head()

,patent_id,citation_id,date
0,5354551,4875247,1989-10-01
1,D674253,D519335,2006-04-01
2,D809697,1963218,1934-06-01
3,D808172,7207426,2007-04-01
4,8683318,6642945,2003-11-01


In [32]:
# citation['patent_year'] = citation['date'].astype(str).str[:4]

In [33]:
# citation = citation.dropna(subset=['patent_year'])

In [34]:
# citation = citation.astype({'year': 'int16'})

In [35]:
# citation = citation[['patent_id','citation_id','year']]

In [36]:
citation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111026439 entries, 0 to 111026438
Data columns (total 3 columns):
patent_id      object
citation_id    object
date           object
dtypes: object(3)
memory usage: 2.5+ GB


In [37]:
# # f_name = "uspc.tsv"
# # f_name = "ipcr.tsv"
# # f_name = "patent.tsv"
# # f_name = "usapplicationcitation.tsv"
# f_name = "uspatentcitation.tsv"
# chunksize = 10 ** 6
# count = 1
# n_obs = 0
# for df in pd.read_csv(f_name, delimiter="\t", chunksize=chunksize, quoting=csv.QUOTE_NONNUMERIC):
#     print('processing chunk: ' + str(count))
#     n_obs += len(df)
#     count += 1
# # Print summary of data: number of observations, columns, and each variable data type
# print(n_obs)
# print(df.dtypes)

In [38]:
# Print summary of data: number of columns, observations, and each variable data type
# print(len(df))
# df.info()
# Print basic summary statistics for numerical variables
# print(df.describe(exclude=[np.number]))

In [39]:
# df.to_csv('usapplicationcitation_sample.csv', index=False)  
# df.to_csv('patent_sample.csv', index=False)
# df.to_csv('uspatentcitation_sample.csv', index=False)
# pd.read_csv('usapplicationcitation_sample.csv')
# pd.read_csv('patent_sample.csv')
# pd.read_csv('uspatentcitation_sample.csv')

In [40]:
# Print first five observations
# df.head(20)
# df.date.max()

## merge datasets

In [41]:
citation.dtypes

patent_id      object
citation_id    object
date           object
dtype: object

In [42]:
nber.dtypes

uuid              object
patent_id         object
category_id         int8
subcategory_id      int8
dtype: object

In [43]:
patent.dtypes

id                        object
type                      object
number                    object
country                   object
date              datetime64[ns]
abstract                  object
title                     object
kind                      object
num_claims                 int16
filename                  object
withdrawn                float32
uuid                      object
patent_id                 object
category_id              float64
subcategory_id           float64
year                       int64
dtype: object

### merge with patents and keep only utility patents

In [44]:
citation.shape[0]

111026439

In [45]:
citation = citation.merge(patent[['id','year','subcategory_id']], left_on = "patent_id", right_on = "id", how = 'left')

In [46]:
citation = citation.rename(columns={"year": "citing_year",'subcategory_id':'citing_subcategory_id'})
citation.sample(5)

,patent_id,citation_id,date,id,citing_year,citing_subcategory_id
1390417,D304944,4691383,1987-09-01,NaN,NaN,NaN
102616876,7070843,5434009,1995-07-01,7070843,2006.0,69.0
24809047,9852658,7241145,2007-07-01,9852658,2017.0,NaN
55872927,7776057,3901244,1975-08-01,7776057,2010.0,32.0
99230076,7674919,6399536,2002-06-01,7674919,2010.0,14.0


# 10% of citations do not have a type..... ok

In [47]:
citation[citation['id'].isnull()].shape[0] / citation.shape[0]

0.10090528076830421

In [48]:
citation = citation.merge(patent[['id','year','subcategory_id']], left_on = "citation_id", right_on = "id", how = 'left')
citation = citation.rename(columns={"year": "cited_year",'subcategory_id':'cited_subcategory_id'})

In [49]:
# 20% of patents don't have a subcategory_id 
citation[citation['cited_subcategory_id'].isnull()].shape[0] / citation.shape[0]

0.2086653432161325

In [50]:
citation

,patent_id,citation_id,date,id_x,citing_year,citing_subcategory_id,id_y,cited_year,cited_subcategory_id
0,5354551,4875247,1989-10-01,5354551,1994.0,31.0,4875247,1989.0,69.0
1,D674253,D519335,2006-04-01,NaN,NaN,NaN,NaN,NaN,NaN
2,D809697,1963218,1934-06-01,NaN,NaN,NaN,NaN,NaN,NaN
3,D808172,7207426,2007-04-01,NaN,NaN,NaN,7207426,2007.0,69.0
4,8683318,6642945,2003-11-01,8683318,2014.0,22.0,6642945,2003.0,22.0
5,10456544,2317608,1943-04-01,10456544,2019.0,NaN,NaN,NaN,NaN
6,8250307,7386701,2008-06-01,8250307,2012.0,24.0,7386701,2008.0,24.0
7,D490798,D190749,1961-06-01,NaN,NaN,NaN,NaN,NaN,NaN
8,9199394,5242647,1993-09-01,9199394,2015.0,NaN,5242647,1993.0,51.0
9,7398575,4894880,1990-01-01,7398575,2008.0,69.0,4894880,1990.0,69.0


### calculate directional citation flows

In [51]:
list_subcat = list(citation['citing_subcategory_id'].dropna().unique())

In [52]:
list_subcat.sort()

In [53]:
df_citation = pd.DataFrame()

for patent_subcategory_id in list_subcat:
    temp = citation[citation['citing_subcategory_id'] == patent_subcategory_id].groupby("cited_subcategory_id").count()\
        [['citing_subcategory_id']].rename(columns={"citing_subcategory_id": patent_subcategory_id})
    df_citation = df_citation.merge(temp, left_index=True, right_index=True, how = "outer")

In [54]:
df_citation = df_citation.replace(np.nan,0)

In [55]:
df_citation.to_csv('citation_cumsum.csv', index=True)